In [7]:
!crest --version


 
       ╔════════════════════════════════════════════╗
       ║            ___ ___ ___ ___ _____           ║
       ║           / __| _ \ __/ __|_   _|          ║
       ║          | (__|   / _|\__ \ | |            ║
       ║           \___|_|_\___|___/ |_|            ║
       ║                                            ║
       ║  Conformer-Rotamer Ensemble Sampling Tool  ║
       ║          based on the xTB methods          ║
       ║                                            ║
       ╚════════════════════════════════════════════╝
       Version 3.0.1, Mon May  6 18:43:33 UTC 2024
       commit (1782d7d) compiled by 'runner@fv-az772-53'
 
 crest 3.0.1


In [8]:
import os, sys
import inspect
from pathlib import Path
import tempfile
from utils import run_command

import rdkit
from rdkit.Chem import AllChem, DataStructs
from rdkit.Chem import AllChem as Chem
from rdkit.Chem import RDConfig
sys.path.append(os.path.join(RDConfig.RDContribDir, 'SA_Score'))
import sascorer

import numpy as np
import torch
import torch.nn as nn

In [9]:
smiles='c1ncc(s1)-c1sc(-c2cnc(s2)-c2scc3cc[se]c23)c2nccnc12'

In [10]:
mol = Chem.MolFromSmiles(smiles)
mol = Chem.AddHs(mol)
charge = Chem.rdmolops.GetFormalCharge(mol)
atom_number = mol.GetNumAtoms()
# Write the smile to a file
system = lambda x: run_command(x, verbose=5)

In [11]:

with open('test.smi', 'w') as f:
    f.write(smiles)

In [12]:

system('obabel test.smi --gen3D -O test.xyz')


1 molecule converted


input_dir = os.path.join(os.getcwd(), 'input_dir')
os.makedirs(input_dir, exist_ok=True)

  # Dossier contenant les fichiers .xyz d'origine
output_dir = 'okay'  # Dossier où seront créés les nouveaux fichiers .xyz
input_file= 'test.xyz'  # Nom du fichier CSV pour le dataframe
max_files = 20885  # Nombre maximum de fichiers à traiter

import subprocess
import os

def run_crest(input_file, output_dir, crest_path='crest'):
    """
    Fonction pour exécuter CREST 3.0 sur un fichier d'entrée et stocker les résultats dans un répertoire de sortie.
    
    :param input_file: Chemin vers le fichier de coordonnées d'entrée (format XYZ).
    :param output_dir: Répertoire où les résultats seront stockés.
    :param crest_path: Chemin vers l'exécutable CREST (défaut est 'crest' si dans le PATH).
    """
    # Assurez-vous que le répertoire de sortie existe
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    # Construire la commande CREST
    command = [crest_path, input_file, '--output', output_dir]
    
    # Exécution de la commande
    try:
        result = subprocess.run(command, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print("CREST a été exécuté avec succès.")
        print("Sortie standard:\n", result.stdout.decode())
        print("Erreur standard:\n", result.stderr.decode())
    except subprocess.CalledProcessError as e:
        print(f"Erreur lors de l'exécution de CREST: {e}")
        print("Sortie standard:\n", e.stdout.decode())
        print("Erreur standard:\n", e.stderr.decode())

if __name__ == '__main__':
    # Exemple d'utilisation
    input_xyz = 'molecule.xyz'  # Chemin vers votre fichier XYZ
    output_directory = 'crest_output'  # Répertoire de sortie
    crest_executable = 'crest'  # Chemin vers l'exécutable CREST si nécessaire

    run_crest(input_xyz, output_directory, crest_executable)

In [6]:

# Run the preliminary xtb: 
command_pre = f'xtb test.xyz --gfn 2 --opt normal --chgr {charge} --iterations 4000'
system(command_pre)

      -----------------------------------------------------------      
     |                   =====================                   |     
     |                           x T B                           |     
     |                   =====================                   |     
     |                         S. Grimme                         |     
     |          Mulliken Center for Theoretical Chemistry        |     
     |                    University of Bonn                     |     
      -----------------------------------------------------------      

   * xtb version 6.6.1 (8d0f1dd) compiled by 'conda@1efc2f54142f' on 2023-08-01

   xtb is free software: you can redistribute it and/or modify it under
   the terms of the GNU Lesser General Public License as published by
   the Free Software Foundation, either version 3 of the License, or
   (at your option) any later version.
   
   xtb is distributed in the hope that it will be useful,
   but WITHOUT ANY WARRANTY; w


           -------------------------------------------------
          |                 G F N 2 - x T B                 |
           -------------------------------------------------

        Reference                      10.1021/acs.jctc.8b01176
      * Hamiltonian:
        H0-scaling (s, p, d)           1.850000    2.230000    2.230000
        zeta-weighting                 0.500000
      * Dispersion:
        s8                             2.700000
        a1                             0.520000
        a2                             5.000000
        s9                             5.000000
      * Repulsion:
        kExp                           1.500000    1.000000
        rExp                           1.000000
      * Coulomb:
        alpha                          2.000000
        third order                    shell-resolved
        anisotropic                    true
        a3                             3.000000
        a5                             4.000000
        cn-

normal termination of xtb
Note: The following floating-point exceptions are signalling: IEEE_UNDERFLOW_FLAG IEEE_DENORMAL


In [13]:

command_crest = f'crest xtbopt.xyz --gfn2 --gbsa h2o --chrg {charge} -T 8'
system(command_crest)

 
       ╔════════════════════════════════════════════╗
       ║            ___ ___ ___ ___ _____           ║
       ║           / __| _ \ __/ __|_   _|          ║
       ║          | (__|   / _|\__ \ | |            ║
       ║           \___|_|_\___|___/ |_|            ║
       ║                                            ║
       ║  Conformer-Rotamer Ensemble Sampling Tool  ║
       ║          based on the xTB methods          ║
       ║                                            ║
       ╚════════════════════════════════════════════╝
       Version 3.0.1, Mon May  6 18:43:33 UTC 2024
       commit (1782d7d) compiled by 'runner@fv-az772-53'
 
   Cite work conducted with this code as

   • P.Pracht, F.Bohle, S.Grimme, PCCP, 2020, 22, 7169-7192.
   • S.Grimme, JCTC, 2019, 15, 2847-2862.
   • P.Pracht, S.Grimme, C.Bannwarth, F.Bohle, S.Ehlert,
     G.Feldmann, J.Gorges, M.Müller, T.Neudecker, C.Plett,
     S.Spicher, P.Steinbach, P.Wesołowski, F.Zeller,
     J. Chem. Phys., 2024, 160, 11

Un changement de topologie a été observé lors de l'optimisation initiale de la géométrie.
Cela pourrait être un artefact du niveau de théorie choisi (par exemple xTB).
Vous pouvez vérifier la trajectoire d'optimisation dans le fichier "crestopt.log".
Essayez l'une de ces options :

stata
Copier
A) Pré-optimisez votre entrée séparément et utilisez la structure optimisée
   comme entrée pour CREST. (Recommandé uniquement si la structure est intacte)

B) Redémarrez le même appel CREST que précédemment, mais ignorez le changement de topologie
   en utilisant le mot-clé "--noreftopo". (Peut produire des artefacts)

C) Corrigez la géométrie d'entrée initiale en introduisant des contraintes de longueur de liaison
   ou en utilisant une méthode avec une topologie fixe (par exemple GFN-FF).

In [28]:

system("rm ./gfnff_charges ./gfnff_topo")

    # Run crest conformer ensemble
command_crest = 'CHARGE={};crest {} -gff -mquick -chrg $CHARGE --noreftopo'.format(charge, 'xtbopt.xyz')
system(command_crest)
system('rm ./gfnff_charges ./gfnff_topo')
system('head -n {} crest_conformers.xyz > crest_best.xyz'.format(atom_number+2))

    # Run the calculation: 
command = 'CHARGE={};xtb {} --opt normal -c $CHARGE --iterations 4000 > out_dump'.format(charge, 'crest_best.xyz')
system(command)

    # Read the output: 
with open('./out_dump', 'r') as f: 
    text_content = f.readlines()


rm: cannot remove './gfnff_charges': No such file or directory
rm: cannot remove './gfnff_topo': No such file or directory


 
       ╔════════════════════════════════════════════╗
       ║            ___ ___ ___ ___ _____           ║
       ║           / __| _ \ __/ __|_   _|          ║
       ║          | (__|   / _|\__ \ | |            ║
       ║           \___|_|_\___|___/ |_|            ║
       ║                                            ║
       ║  Conformer-Rotamer Ensemble Sampling Tool  ║
       ║          based on the xTB methods          ║
       ║                                            ║
       ╚════════════════════════════════════════════╝
       Version 3.0.1, Mon May  6 18:43:33 UTC 2024
       commit (1782d7d) compiled by 'runner@fv-az772-53'
 
   Cite work conducted with this code as

   • P.Pracht, F.Bohle, S.Grimme, PCCP, 2020, 22, 7169-7192.
   • S.Grimme, JCTC, 2019, 15, 2847-2862.
   • P.Pracht, S.Grimme, C.Bannwarth, F.Bohle, S.Ehlert,
     G.Feldmann, J.Gorges, M.Müller, T.Neudecker, C.Plett,
     S.Spicher, P.Steinbach, P.Wesołowski, F.Zeller,
     J. Chem. Phys., 2024, 160, 11

rm: cannot remove './gfnff_charges': No such file or directory
normal termination of xtb
Note: The following floating-point exceptions are signalling: IEEE_UNDERFLOW_FLAG IEEE_DENORMAL
